# CA02
This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

## Import Packages

In [0]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

## Functions

This function builds a Dictionary of most common 3000 words from all the email content. First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.

In [0]:
def make_Dictionary(root_dir):
  all_words = []
  # Join all the file(f) in the directory "root_dir" to the path "root_dir"
  # "emails" is now a list of path of the txt files
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  for email in emails:
    # open each txt file
    with open(email) as m:
      for line in m:
        # seperate each line to words
        words = line.split()
        # Use += instead of directly appending to "all_words" because a "words" is a list of words in a line,
        # therefore, it would append multiple lists to "all_words"
        all_words += words
  # Counter is a function that counts each element in a list
  valid_word = Counter(all_words)
  # A list of the unique words in the Counter
  list_to_remove = list(valid_word)

  for item in list_to_remove:
    # isalpha() is a method that check if the argument contains any alphabets characters
    if item.isalpha() == False:
      # delete the words that are not alphabets characters
      del valid_word[item]
    elif len(item) == 1:
      # delete the words that equal to 1 character
      del valid_word[item]
  # a list of the n most common elements and their counts from the most common to the least.
  valid_word = valid_word.most_common(3000)
  return valid_word

This function extracts feature columns and populates their values (Feature Matrix of 3000 columms and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [0]:
def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  docID = 0;
  for fil in files:
    with open(fil) as fi:
      # Give each line of text an index -> 'i' is the index starting from 0 and 'line' is the element
      # ex. [(0,"subject"),(1,"blank"),(2,"content")]
      for i, line in enumerate(fi):
        # the content is in the third element which the index==2
        if i ==2:
          # split the line to words
          words = line.split()
          for word in words:
            # set the wordID as an integer 0
            wordID = 0
            # Give each word in the valid_word an index -> 'i' is the index starting from 0 and 'd' is the word
            # ex. [(0,"word1"),(1,"word2").....(2999,"word2999")]
            for i, d in enumerate(valid_word):
              # d[0] is the word in valid_word and d[1] is the count of the word
              # so if "word" have shown up in valid_word which is d[0] then:
              if d[0] == word:
                # the wordID would be the index of the word in valid_word
                wordID = i
                # let's assume we are dealing with the first document and the "word" in "words" have all shown up in a enumerate(valid_word)
                # that looks like [(0,'to'),(1,'be'),(2,'or'),(3,'not')]
                # -> the parameters would be: docID = 0, wordID=0, words=['to','be','or','not','to','be']
                # -> feature_matrix[0,0]=words.count('to'), feature_matrix[0,1]=words.count('be'),
                #    feature_matrix[0,3]=words.count('or'), feature_matrix[0,4]=words.count('not')
                # ->            0(to) 1(be) 2(or) 3(not) 
                #   0(1st doc) [2,    2,    1,    1]
                # * It's the frequency of each word in valid_word in nth document.*
                features_matrix[docID,wordID] = words.count(word)
      # train_labels[docID] = 0; # train_labels is already set up as a np.zero array
      # split the "fil" by "/"
      filepathTokens = fil.split('/')
      # Python's index start from 0, so the len()-1 would be the index of last token
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
      # jump to the next document
      docID = docID + 1
  return features_matrix, train_labels

## Main Program

The section is the main Program that calls the above two functions and gets executed first. First it "trains" the model using model.fit function and Training Dataset. After that it scores the Test Data set by running the Trained Model with the Test Data set. At the end it prints the model performance in terms of accuracy score.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
TRAIN_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails'
TEST_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/test-mails'

print ("Find out all the valid and unique words in the TRAIN folder")
valid_word = make_Dictionary(TRAIN_DIR)

print ("Reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

model = GaussianNB()

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
# X: features_matrix, y: labels
model.fit(features_matrix, labels)
print ("Training completed")
print ("Testing trained model to predict Test Data labels")
# put the X_test to the model for prediction
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... ")
print ("Now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
# inspect the accuracy by checking the real X_label with predicted_X_label
print (accuracy_score(test_labels, predicted_labels))

Find out all the valid and unique words in the TRAIN folder
Reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
Testing trained model to predict Test Data labels
Completed classification of the Test Data .... 
Now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616
